### Importando biblíotecas necessárias

In [274]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [275]:
dfBooks = pd.read_csv('archive/Books.csv')

C:\Users\Gabriel Lopes\AppData\Local\Temp\ipykernel_21084\2782996754.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  dfBooks = pd.read_csv('archive/Books.csv')


In [276]:
dfUsers = pd.read_csv('archive/Users.csv')

In [277]:
dfRatings = pd.read_csv('archive/Ratings.csv')

### Primeiras impressões dos dados

In [278]:
dfUsers.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [279]:
dfBooks.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [280]:
dfRatings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [281]:
dfBooks.rename(columns={'User-ID': 'user_id', 'Book-Author': 'author', 'Year-Of-Publication': 'year', 'Publisher': 'publisher', 'Book-Title': 'title'}, inplace=True)

In [282]:
dfRatings.rename(columns={'User-ID': 'user_id', 'Book-Rating': 'rating'}, inplace=True)

In [283]:
dfRatings.head(5)

,user_id,ISBN,rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [284]:
dfRatings.shape

(1149780, 3)

### Verificando a frequência de avalições dos usuários

In [285]:
count = dfRatings['user_id'].value_counts()

In [286]:
count

user_id
11676     13602
198711     7550
153662     6109
98391      5891
35859      5850
          ...  
116180        1
116166        1
116154        1
116137        1
276723        1
Name: count, Length: 105283, dtype: int64

Podemos perceber que há usuários que avaliam bem menos livros **e outros que mantem um hábito de leitura bem maior, logo avaliam mais livros. são esses que queremos usar para pegar de parametro para o modelo.**

In [287]:
dfRatings = dfRatings[dfRatings['user_id'].isin(count[count > 200].index)]

In [288]:
dfRatings.shape

(526356, 3)

**Metade do dataframe fo eliminado fazendo esse filtro dos usuários que avaliaram mais que 200 livros.**

In [289]:
rating_books = dfRatings.merge(dfBooks, on='ISBN')

In [290]:
rating_books.shape

(487671, 10)

In [291]:
num_rating = rating_books.groupby('title')['rating'].count().reset_index()

Contando a quantidade total de avaliações por cada titulo de livro.

In [292]:
num_rating.rename(columns={'rating': 'num_rating'}, inplace=True)

In [293]:
num_rating.shape

(160269, 2)

In [294]:
final_data = rating_books.merge(num_rating, on='title')

In [295]:
final_data.shape

(487671, 11)

In [296]:
final_data = final_data[final_data['num_rating'] >= 50]

Esta linha de comando seleciona apenas os livros que tem mais de 50 avaliações. **Isso filtrou cerca de 83% do dataset.** Caso não fosse aplicado esse filtro o modelo iria se perder dos livros mais importantes em meio a quantidade imensa com baixa quantidade de avaliação.

In [297]:
final_data.shape

(61853, 11)

In [298]:
final_data = final_data.drop_duplicates(['user_id', 'title'])

Dropando os dados duplicados.

In [299]:
final_data.shape

(59850, 11)

In [300]:
final_data.head(2)

,user_id,ISBN,rating,title,author,year,publisher,Image-URL-S,Image-URL-M,Image-URL-L,num_rating
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,82
13,277427,0060930535,0,The Poisonwood Bible: A Novel,Barbara Kingsolver,1999,Perennial,http://images.amazon.com/images/P/0060930535.0...,http://images.amazon.com/images/P/0060930535.0...,http://images.amazon.com/images/P/0060930535.0...,133


In [301]:
final_data.drop(columns=['Image-URL-S', 'Image-URL-M'], inplace=True)

In [302]:
final_data.shape

(59850, 9)

In [303]:
final_data.rename(columns={'Image-URL-L': 'image_url'}, inplace=True)

In [304]:
final_data.head(2)

,user_id,ISBN,rating,title,author,year,publisher,image_url,num_rating
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
13,277427,0060930535,0,The Poisonwood Bible: A Novel,Barbara Kingsolver,1999,Perennial,http://images.amazon.com/images/P/0060930535.0...,133


In [305]:
book_pivot = final_data.pivot_table(columns='user_id', index='title', values='rating').fillna(0)

In [306]:
book_pivot

user_id,254,2276,2766,2977,3363,3757,4017,4385,6242,6251,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
84 Charing Cross Road,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,7.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
You Belong To Me,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


A função pivot_table do pandas é frequentemente usada em sistemas de recomendação para transformar dados de formato longo (onde cada linha representa uma interação entre um usuário e um item) em uma matriz de utilidade (ou matriz de interação), onde as linhas representam usuários, as colunas representam itens, e os valores representam as interações (como classificações, visualizações, etc.).

Essa matriz de utilidade é essencial para muitos algoritmos de recomendação, especialmente aqueles baseados em filtragem colaborativa, como o K-Nearest Neighbors (KNN) e a decomposição em valores singulares (SVD).

In [307]:
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import mean_squared_error

A função csr_matrix do módulo scipy.sparse é usada para criar uma matriz esparsa no formato CSR (Compressed Sparse Row). Esse formato é eficiente em termos de memória e operações de matriz, especialmente quando a matriz contém muitos zeros, o que é comum em sistemas de recomendação.

Vantagens do Formato CSR

- Eficiência de Memória: Armazena apenas os elementos não-zero, economizando memória.

- Operações Rápidas: Permite operações eficientes de fatiamento e aritmética de matriz.

- Conversão Fácil: Pode ser facilmente convertida para outros formatos esparsos.

In [308]:
# transpor a matriz para que os usuarios sejam as linhas e as colunas os livros
book_pivot = book_pivot.T

In [309]:
svd = TruncatedSVD(n_components=2)
matrix_svd = svd.fit_transform(book_pivot)

In [310]:
reconstructed_matrix = np.dot(matrix_svd, svd.components_)

In [311]:
reconstructed_matrix

array([[0.28896418, 0.35223408, 0.29416407, ..., 0.22200291, 0.15037373,
        0.20059534],
       [0.11707273, 0.2618288 , 0.23324033, ..., 0.10998458, 0.10704226,
        0.17187792],
       [0.29165273, 0.30216727, 0.24582361, ..., 0.21509395, 0.1311204 ,
        0.16188694],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.04563357, 0.23372836, 0.21699011, ..., 0.06502271, 0.09270078,
        0.16714784],
       [0.04104728, 0.13492426, 0.12306839, ..., 0.0458171 , 0.05422597,
        0.09306354]])

In [312]:
rmse = np.sqrt(mean_squared_error(book_pivot, reconstructed_matrix))
print(f"RMSE: {rmse}")

RMSE: 1.1649361764450092


In [313]:
from sklearn.neighbors import NearestNeighbors

**Como Funciona o Algoritmo NearestNeighbors** <br><br>
O algoritmo NearestNeighbors pode ser configurado para usar diferentes métricas de distância e algoritmos de busca para encontrar os vizinhos mais próximos.

Métricas de Distância:

- euclidean: Distância Euclidiana.
- manhattan: Distância de Manhattan (ou L1).
- cosine: Distância do Cosseno.
- Outras métricas personalizadas.

Algoritmos de Busca:

- brute: Busca exaustiva (força bruta).
- kd_tree: Árvore KD (K-Dimensional).
- ball_tree: Árvore de Bolas.
- auto: Seleciona automaticamente o melhor algoritmo com base nos dados.
- Número de Vizinhos:

n_neighbors: Número de vizinhos mais próximos a serem encontrados.

In [314]:
model = NearestNeighbors(algorithm='auto', metric='cosine')

In [315]:
model.fit(reconstructed_matrix)

NearestNeighbors(metric='cosine')

In [316]:
user_index = 237

In [317]:
distance, suggestions = model.kneighbors(reconstructed_matrix[user_index].reshape(1, -1), n_neighbors=6)


Análise da Função `model.kneighbors(...)`

Descrição: Esta função faz parte da classe `NearestNeighbors` do **scikit-learn** e é usada para identificar os vizinhos mais próximos de um ponto de consulta. O `model` é uma instância previamente treinada da classe `NearestNeighbors`.

Detalhes de `book_pivot.iloc[237, :]`:

- Contexto: `book_pivot` é um **DataFrame**, provavelmente uma matriz de utilidade onde:  
  - Linhas representam livros.  
  - Colunas representam usuários.  

- Seleção: `iloc[237, :]` seleciona a **linha 237** do DataFrame `book_pivot`. Essa linha contém as características do livro com índice **237**.

Transformação com `.values.reshape(1, -1)`:

- `.values`: Converte a linha selecionada em um **array NumPy**.  
- `.reshape(1, -1)`: Reorganiza o array em uma **matriz 2D** com:  
  - Uma única linha.  
  - Colunas suficientes para acomodar todos os elementos (-1 indica que o número de colunas é calculado automaticamente).  

  Essa transformação é necessária porque a função `kneighbors` requer que a entrada seja uma **matriz 2D**.

Parâmetro `n_neighbors=6`:

- Indica que a busca será feita para encontrar os **6 vizinhos mais próximos** do ponto de consulta.


In [318]:
distance

array([[0.00000000e+00, 1.56920523e-05, 2.26935095e-05, 2.78320502e-05,
        5.07023540e-05, 5.92138714e-05]])

In [319]:
suggestions

array([[237,  47, 188, 407, 734, 662]], dtype=int64)

**Medindo o desempenho do modelo**

Entrada: O código pega as características do livro na linha 237 do DataFrame book_pivot.

Processamento: Usa o modelo NearestNeighbors para encontrar os 6 livros mais próximos (vizinhos) ao livro na linha 237, com base nas características fornecidas.

Saída: A função kneighbors retorna duas coisas:

distance: Um array contendo as distâncias dos 6 livros mais próximos ao livro de consulta.

suggestions: Um array contendo os índices dos 6 livros mais próximos no DataFrame book_pivot.

In [320]:
for i in range(len(suggestions)):
    print(book_pivot.index[suggestions[i]])

Index([85701, 18401, 69232, 135045, 234597, 211426], dtype='int64', name='user_id')


In [321]:
books_name = book_pivot.index

In [322]:
books_name

Index([   254,   2276,   2766,   2977,   3363,   3757,   4017,   4385,   6242,
         6251,
       ...
       274004, 274061, 274301, 274308, 274808, 275970, 277427, 277478, 277639,
       278418],
      dtype='int64', name='user_id', length=888)


Exemplo prático:

Suponha que:

- `suggestions` seja `[237, 42, 15, 89, 300]`, ou seja, uma lista com 5 índices de livros que são "sugeridos" como similares (talvez encontrados pelo `kneighbors`).  
- `book_pivot.index` seja uma lista como `["O Senhor dos Anéis", "Harry Potter", "Código Da Vinci", ...]`, onde cada posição corresponde a um livro.

O loop fará o seguinte:

- Primeira iteração (`i = 0`):  
  `print(book_pivot.index[suggestions[0]])` imprime o livro na posição `suggestions[0]` (237). Se `book_pivot.index[237]` for "O Senhor dos Anéis", a saída será:  
  `O Senhor dos Anéis`

- Segunda iteração (`i = 1`):  
  `print(book_pivot.index[suggestions[1]])` imprime o livro na posição `suggestions[1]` (42). Se `book_pivot.index[42]` for "Harry Potter", a saída será:  
  `Harry Potter`

E assim por diante, até que todos os índices em `suggestions` sejam processados. No final, o código simplesmente lista os títulos ou identificadores dos livros que foram considerados "sugestões" ou "vizinhos mais próximos".


In [323]:
import pickle

In [324]:
# pickle.dump(books_name, open('artefatos/book_name.pkl', 'wb'))
# pickle.dump(model, open('artefatos/model.pkl', 'wb'))
# pickle.dump(final_data, open('artefatos/final_data.pkl', 'wb'))
# pickle.dump(book_pivot, open('artefatos/book_pivot.pkl', 'wb'))

In [325]:
def evaluate_recommendations(book_title, model, reconstructed_matrix, book_pivot, k=6):
    # Verifique se o livro está no índice de book_pivot
    if book_title not in book_pivot.index:
        print(f"Erro: O livro '{book_title}' não foi encontrado na base de dados.")
        return 0, 0, []  # Retorna valores padrão caso o livro não seja encontrado
    
    # Encontre o ID do livro
    book_id = np.where(book_pivot.index == book_title)[0][0]
    
    # Obtenha recomendações
    distances, suggestions = model.kneighbors(reconstructed_matrix[book_id].reshape(1, -1), n_neighbors=k)
    recommended_books = book_pivot.index[suggestions[0]]
    print(f"Recomendações para '{book_title}':", recommended_books.tolist())
    
    # Encontre livros relevantes (baseado nas avaliações internas)
    # Aqui, estamos assumindo que livros com avaliações altas são aqueles com maior média de avaliações
    mean_ratings = book_pivot.mean(axis=1)  # Média de avaliações por livro
    true_likes = book_pivot.index[mean_ratings > mean_ratings.quantile(0.75)].tolist()  # Top 25% de avaliações
    true_likes = [book for book in true_likes if book != book_title]  # Exclui o próprio livro
    
    # Calcule acertos
    hits = len(set(recommended_books) & set(true_likes))
    precision = hits / k if k > 0 else 0
    recall = hits / len(true_likes) if true_likes else 0
    
    return precision, recall, true_likes

In [326]:
# Teste para um livro
book_title = "1984"
precision, recall, true_likes = evaluate_recommendations(book_title, model, reconstructed_matrix, book_pivot)

if precision is not None:  # Só imprime se o livro foi encontrado
    print(f"Precisão@{6}: {precision}, Recall@{6}: {recall}")
    print(f"Livros relevantes reais: {true_likes}")

Erro: O livro '1984' não foi encontrado na base de dados.
Precisão@6: 0, Recall@6: 0
Livros relevantes reais: []


In [327]:
books_to_test = ["1984", "1st to Die: A Novel", "2nd Chance", "Livro Inexistente"]  # Adicionei um livro que não existe para teste
precisions, recalls = [], []
for book in books_to_test:
    p, r, _ = evaluate_recommendations(book, model, reconstructed_matrix, book_pivot)
    if p is not None:  # Só adiciona se o livro foi encontrado
        precisions.append(p)
        recalls.append(r)

if precisions:  # Só imprime se pelo menos um livro foi encontrado
    print(f"Média Precisão@6: {np.mean(precisions)}, Média Recall@6: {np.mean(recalls)}")

Erro: O livro '1984' não foi encontrado na base de dados.
Erro: O livro '1st to Die: A Novel' não foi encontrado na base de dados.
Erro: O livro '2nd Chance' não foi encontrado na base de dados.
Erro: O livro 'Livro Inexistente' não foi encontrado na base de dados.
Média Precisão@6: 0.0, Média Recall@6: 0.0


In [328]:
book_id = np.where(book_pivot.index == 'Jacob Have I Loved')[0][0]
distance, suggestions = model.kneighbors(reconstructed_matrix.iloc[book_id].reshape(1, -1), n_neighbors=6)

IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
print(suggestions)

[[291 184 523 269   8  39]]


In [ ]:
def recomendation(book_title, model, reconstructed_matrix, book_pivot):
    book_id = np.argwhere(book_pivot.index == book_title)[0][0]
    distance, suggestions = model.kneighbors(reconstructed_matrix[book_id].reshape(1, -1), n_neighbors=6)
    recomemended = book_pivot.index(suggestions[0])
    return recomemended.tolist()

In [ ]:
book_title = 'Jacob Have I Loved'
recomendation(book_title)

Jacob Have I Loved
Exclusive
The Burden of Proof
I Know Why the Caged Bird Sings
A Civil Action
Alaska


In [ ]:
sum_rating = final_data[final_data['title'] == book_title]
sum_rating['rating'].sum()  

33